# Tema 2 - IA
##### Alexe Alexandra Florentina  - 333CC

# Clasificare diabet
## 3.1. Explorarea datelor
### 3.1.1. Analiza tipului de atribute și a plajei de valori a acestora


In [ ]:
import pandas as pd

# Citirea fișierului CSV
data = pd.read_csv("tema2_Diabet/diabet_full.csv")

num_attributes = ['psychological-rating', 'BodyMassIndex', 'Age', 'CognitionScore', 'Body_Stats', 'Metabolical_Rate']


# Afișarea statisticilor generale despre coloanele numerice
print(data['psychological-rating'].describe())
print(data['BodyMassIndex'].describe())
print(data['Age'].describe())
print(data['CognitionScore'].describe())
print(data['Body_Stats'].describe())
print(data['Metabolical_Rate'].describe())



In [ ]:
import matplotlib.pyplot as plt

# Trasarea boxplot-urilor pentru fiecare atribut numeric
for col in num_attributes:
    data[[col]].boxplot(figsize=(8, 6))
    plt.title(f'Boxplot pentru atributul {col}')
    plt.ylabel('Valoare')
    plt.xlabel('Atribut')
    plt.show()

In [ ]:
import pandas as pd
from tabulate import tabulate

# Tabel cu date pt fiecare atribut discret sau ordinal

# Coloanele de interes
disc_ord_attributes = ['HealthcareInterest', 'PreCVA', 'RoutineChecks', 'CompletedEduLvl', 'alcoholAbuse', 'cholesterol_ver',
                'vegetables', 'HighBP', 'Unprocessed_fructose', 'Jogging', 'IncreasedChol', 'gender', 'HealthScore',
                'myocardial_infarction', 'SalaryBraket', 'Cardio', 'ImprovedAveragePulmonaryCapacity', 'Smoker', 'Diabetes']


# Inițializez o listă pentru a stoca rezultatele
rezultate = []

for coloana in disc_ord_attributes:
    num_fara_lipsa = data[coloana].notna().sum()  # Numărul de exemple fără valori lipsă
    num_valori_unice = len(pd.unique(data[coloana]))  # Numărul de valori unice

    # Adăug rezultatele în listă sub forma de tuplu
    rezultate.append((coloana, num_fara_lipsa, num_valori_unice))

# Afișez rezultatele sub forma unui tabel
print(tabulate(rezultate, headers=["Coloana", "Numărul de exemple fără valori lipsă", "Numărul de valori unice"], tablefmt="pretty"))

In [ ]:

# Trasarea histogramelor pentru fiecare atribut discret sau ordinal

for coloana in disc_ord_attributes :
    # Obțin distribuția valorilor pentru atributul curent
    distributie_valori = data[coloana].value_counts()

    # Creez histograma
    plt.figure(figsize=(8, 6))  
    ax = distributie_valori.plot(kind='bar', color='skyblue')  # Folosim un grafic de tip bar pentru histogramă
    plt.title(f'Distribuția valorilor pentru {coloana}')  
    plt.xlabel('Valoare')  
    plt.ylabel('Frecvență')  
    # Rotesc etichetele de pe axa x pentru o citire mai ușoară
    plt.xticks(rotation=45)  
    
    # Adaug numărul de valori de fiecare tip în partea de sus a fiecărei bare
    for i, valoare in enumerate(distributie_valori):
        ax.text(i, valoare, str(valoare), ha='center', va='bottom')

    plt.tight_layout()  
    plt.show()  

### 3.1.2. Analiza echilibrului de clase

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Citirea fișierului CSV
data_test = pd.read_csv("tema2_Diabet/diabet_test.csv")
data_train = pd.read_csv("tema2_Diabet/diabet_train.csv")

disc_ord_attributes  = ['HealthcareInterest', 'PreCVA', 'RoutineChecks', 'CompletedEduLvl', 'alcoholAbuse', 'cholesterol_ver',
                'vegetables', 'HighBP', 'Unprocessed_fructose', 'Jogging', 'IncreasedChol', 'gender', 'HealthScore',
                'myocardial_infarction', 'SalaryBraket', 'Cardio', 'ImprovedAveragePulmonaryCapacity', 'Smoker', 'Diabetes']

# Parcurge fiecare atribut și realizează graficul pentru fiecare
for atribut in disc_ord_attributes:
    # Creează un DataFrame combinat pentru valorile atributului din seturile de date de antrenare și de testare
    all_values = pd.concat([data_train[[atribut]], data_test[[atribut]]], ignore_index=True)
    
    # Afișează distribuția valorilor pentru atributul curent
    plt.figure(figsize=(10, 6))
    sns.countplot(data=all_values, x=atribut, hue=all_values.index // len(data_train))
    plt.title(f'Distribuția valorilor pentru {atribut} în seturile de date de antrenare și de testare')
    plt.xlabel('Valoare atribut')
    plt.ylabel('Număr de exemple')
    
    # Adăugarea numerelor exacte deasupra barelor
    for p in plt.gca().patches:
        plt.gca().annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                           ha='center', va='center', fontsize=8, color='black', xytext=(0, 5), 
                           textcoords='offset points')
    
    plt.legend(title='Set de date', labels=['Antrenare', 'Testare'])
    plt.show()


### 3.1.3. Analiza corelatiei intre atribute
- atributele numerice

In [ ]:
correlation_matrix = data[num_attributes].corr(method='pearson')
# Vizualizarea matricei de corelație
plt.figure(figsize=(10, 8))
plt.matshow(correlation_matrix, fignum=1, cmap='coolwarm')
plt.colorbar()
plt.xticks(range(len(num_attributes)), num_attributes, rotation=45, ha='left')
plt.yticks(range(len(num_attributes)), num_attributes)
plt.title('Matricea de corelație între atributele numerice continue', pad=20)
plt.show()

- atribute categorice 

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Datele categorice
categorical_data = data[disc_ord_attributes]

# Crearea unui DataFrame gol pentru stocarea rezultatelor testului Chi-Pătrat
chi2_results = pd.DataFrame(index=disc_ord_attributes, columns=disc_ord_attributes)

# Calcularea testului Chi-Pătrat pentru fiecare pereche de atribute categorice
for col1 in disc_ord_attributes:
    for col2 in disc_ord_attributes:
        if col1 != col2:
            contingency_table = pd.crosstab(categorical_data[col1], categorical_data[col2])
            chi2, _, _, _ = chi2_contingency(contingency_table)
            chi2_results.at[col1, col2] = chi2

# Afișarea rezultatelor
print("Matricea de corelație Chi-Pătrat între atributele categorice:")
print(chi2_results)



# II. Clasificare risc de creditare
## 3.1. Explorarea datelor
### 3.1.1. Analiza tipului de atribute și a plajei de valori a acestora

In [ ]:
# Citirea fișierului CSV
data2 = pd.read_csv("tema2_Credit_Risk/credit_risk_full.csv")

num_attributes2 = ['applicant_age', 'applicant_income', 'job_tenure_years', 'loan_amount', 'loan_rate', 'loan_income_ratio', 'credit_history_length_years', 'credit_history_length_months']


# Afișarea statisticilor generale despre coloanele numerice
print(data2['applicant_age'].describe())
print(data2['applicant_income'].describe())
print(data2['job_tenure_years'].describe())
print(data2['loan_amount'].describe())
print(data2['loan_rate'].describe())
print(data2['loan_income_ratio'].describe())
print(data2['credit_history_length_years'].describe())
print(data2['credit_history_length_months'].describe())

In [ ]:
# Trasarea boxplot-urilor pentru fiecare atribut numeric
for col in num_attributes2:
    data2[[col]].boxplot(figsize=(8, 6))
    plt.title(f'Boxplot pentru atributul {col}')
    plt.ylabel('Valoare')
    plt.xlabel('Atribut')
    plt.show()

In [ ]:
# Tabel cu date pt fiecare atribut discret sau ordinal

# Coloanele de interes
disc_ord_attributes2 = ['residential_status', 'loan_purpose', 'loan_rating', 'credit_history_default_status', 'stability_rating', 'loan_approval_status']


rezultate = []

for coloana in disc_ord_attributes2:
    num_fara_lipsa = data2[coloana].notna().sum()  # Numărul de exemple fără valori lipsă
    num_valori_unice = len(pd.unique(data2[coloana]))  # Numărul de valori unice

    # Adăug rezultatele în listă sub forma de tuplu
    rezultate.append((coloana, num_fara_lipsa, num_valori_unice))

# Afișez rezultatele sub forma unui tabel
print(tabulate(rezultate, headers=["Coloana", "Numărul de exemple fără valori lipsă", "Numărul de valori unice"], tablefmt="pretty"))

In [ ]:
# Trasarea histogramelor pentru fiecare atribut discret sau ordinal
for coloana in disc_ord_attributes2 :
    # Obțin distribuția valorilor pentru atributul curent
    distributie_valori = data2[coloana].value_counts()

    # Creez histograma
    plt.figure(figsize=(8, 6))  
    ax = distributie_valori.plot(kind='bar', color='skyblue')  # Folosim un grafic de tip bar pentru histogramă
    plt.title(f'Distribuția valorilor pentru {coloana}')  
    plt.xlabel('Valoare')  
    plt.ylabel('Frecvență')  
    # Rotesc etichetele de pe axa x pentru o citire mai ușoară
    plt.xticks(rotation=45)  
    
    # Adaug numărul de valori de fiecare tip în partea de sus a fiecărei bare
    for i, valoare in enumerate(distributie_valori):
        ax.text(i, valoare, str(valoare), ha='center', va='bottom')

    plt.tight_layout()  
    plt.show()  

### 3.1.2. Analiza echilibrului de clase

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Citirea fișierului CSV
data_test2 = pd.read_csv("tema2_Credit_Risk/credit_risk_test.csv")
data_train2 = pd.read_csv("tema2_Credit_Risk/credit_risk_train.csv")

# Parcurge fiecare atribut și realizează graficul pentru fiecare
for atribut in disc_ord_attributes2:
    # Creează un DataFrame combinat pentru valorile atributului din seturile de date de antrenare și de testare
    all_values = pd.concat([data_train2[[atribut]], data_test2[[atribut]]], ignore_index=True)
    
    # Afișează distribuția valorilor pentru atributul curent
    plt.figure(figsize=(10, 6))
    sns.countplot(data=all_values, x=atribut, hue=all_values.index // len(data_train2))
    plt.title(f'Distribuția valorilor pentru {atribut} în seturile de date de antrenare și de testare')
    plt.xlabel('Valoare atribut')
    plt.ylabel('Număr de exemple')
    
    # Adăugarea numerelor exacte deasupra barelor
    for p in plt.gca().patches:
        plt.gca().annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                           ha='center', va='center', fontsize=8, color='black', xytext=(0, 5), 
                           textcoords='offset points')
    
    plt.legend(title='Set de date', labels=['Antrenare', 'Testare'])
    plt.show()

### 3.1.3. Analiza corelatiei intre atribute
- atributele numerice

In [ ]:
correlation_matrix2 = data[num_attributes2].corr(method='pearson')
# Vizualizarea matricei de corelație
plt.figure(figsize=(10, 8))
plt.matshow(correlation_matrix2, fignum=1, cmap='coolwarm')
plt.colorbar()
plt.xticks(range(len(num_attributes2)), num_attributes2, rotation=45, ha='left')
plt.yticks(range(len(num_attributes2)), num_attributes2)
plt.title('Matricea de corelație între atributele numerice continue', pad=20)
plt.show()

- atribute categorice

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Datele categorice
categorical_data = data2[disc_ord_attributes2]

# Crearea unui DataFrame gol pentru stocarea rezultatelor testului Chi-Pătrat
chi2_results = pd.DataFrame(index=disc_ord_attributes2, columns=disc_ord_attributes2)

# Calcularea testului Chi-Pătrat pentru fiecare pereche de atribute categorice
for col1 in disc_ord_attributes2:
    for col2 in disc_ord_attributes2:
        if col1 != col2:
            contingency_table = pd.crosstab(categorical_data[col1], categorical_data[col2])
            chi2, _, _, _ = chi2_contingency(contingency_table)
            chi2_results.at[col1, col2] = chi2

# Afișarea rezultatelor
print("Matricea de corelație Chi-Pătrat între atributele categorice:")
print(chi2_results)
